In [1]:
from datasets import load_dataset, Dataset, load_from_disk
from jinja2 import Template
from transformers import AutoTokenizer
from vllm import LLM, SamplingParams

import json
import re

INFO 03-19 01:45:06 [__init__.py:256] Automatically detected platform cuda.


In [2]:
NUM_GPUS = 4
BATCH_SIZE = 256

In [3]:
ds = load_dataset('amang1802/wildeweb_cls_1M')['train']

In [4]:
ds

Dataset({
    features: ['text', 'id', 'dump', 'url', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score', 'justification', 'classification_score'],
    num_rows: 1000000
})

In [5]:
system_prompt = """
Below is an extract from a web page. You are an AI content evaluator focused on assessing educational material's value for soft skills development. Soft skills include conversational ability, empathy, leadership skills, public speaking, confidence building, critical thinking, problem solving, professional writing, teamwork, digital literacy, professional attitude, work ethic, career management and intercultural fluency. 

You will analyze content using the additive 5-point scoring system described below. Points are accumulated based on the satisfaction of each criterion:
- Add 1 point if the extract hows superficial coverage of basic communication and teamwork concepts without meaningful depth or practical application. Professional development opportunities are limited to theoretical knowledge, and problem-solving scenarios lack complexity or real-world context. Cultural awareness and digital literacy elements are either absent or extremely basic.
- Add another point if the extract specifically includes discussion of soft skills and includes straightforward communication scenarios and simple team dynamics, but lacks nuanced interaction or complex problem-solving opportunities. Professional development focuses on fundamental skills with limited practical application, while cultural awareness and digital literacy are present but superficial.
- Award a third point if the extract specifically includes discussion of soft skills andfeatures realistic scenarios that integrate emotional intelligence, leadership challenges, and critical thinking opportunities. Professional development includes practical applications with meaningful context, while incorporating cultural awareness and modern digital literacy skills throughout the material. 
- Grant a fourth point if the extract specifically includes discussion of soft skills and presents complex scenarios requiring sophisticated communication, strategic thinking, and advanced problem-solving across multiple contexts. Professional development opportunities are comprehensive and practical, with strong emphasis on intercultural fluency and technological adaptation.
- Bestow a fifth point if the extract specifically includes discussion of soft skills and seamlessly integrates advanced communication, leadership, and problem-solving scenarios that mirror real-world complexity. Professional development opportunities span multiple contexts with sophisticated cultural awareness, while digital literacy and practical application are woven throughout every element.

After examining the extract: 
- Briefly justify your total score, up to 100 words.
- Conclude with the score using the format: "Educational score: <total points>"
"""

In [6]:
model_id = "google/gemma-3-27b-it"

In [7]:
llm = LLM(model=model_id, max_model_len=16384, tensor_parallel_size=NUM_GPUS, gpu_memory_utilization=0.98)

INFO 03-19 01:45:07 [config.py:2537] For Gemma 2 and 3, we downcast float32 to bfloat16 instead of float16 by default. Please specify `dtype` if you want to use float16.
INFO 03-19 01:45:07 [config.py:2595] Downcasting torch.float32 to torch.bfloat16.
INFO 03-19 01:45:14 [config.py:583] This model supports multiple tasks: {'embed', 'reward', 'generate', 'classify', 'score'}. Defaulting to 'generate'.
WARNING 03-19 01:45:14 [arg_utils.py:1754] ['Gemma3ForConditionalGeneration'] is not supported by the V1 Engine. Falling back to V0. 
INFO 03-19 01:45:14 [config.py:1515] Defaulting to use mp for distributed inference
INFO 03-19 01:45:14 [llm_engine.py:241] Initializing a V0 LLM engine (v0.8.0) with config: model='google/gemma-3-27b-it', speculative_config=None, tokenizer='google/gemma-3-27b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16384, download_dir=

[W319 01:45:20.527957295 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W319 01:45:20.528419069 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W319 01:45:20.705683585 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W319 01:45:20.706057837 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W319 01:45:20.726162545 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W319 01:45:20.726377189 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 03-19 01:45:20 [utils.py:925] Found nccl from library libnccl.so.2
INFO 03-19 01:45:20 [pynccl.py:69] vLLM is using nccl==2.21.5
(VllmWorkerProcess pid=1046555) (VllmWorkerProcess pid=1046552) (VllmWorkerProcess pid=1046560) INFO 03-19 01:45:20 [utils.py:925] Found nccl from library libnccl.so.2
INFO 03-19 01:45:20 [utils.py:925] Found nccl from library libnccl.so.2
INFO 03-19 01:45:20 [utils.py:925] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=1046555) (VllmWorkerProcess pid=1046552) INFO 03-19 01:45:20 [pynccl.py:69] vLLM is using nccl==2.21.5
(VllmWorkerProcess pid=1046560) INFO 03-19 01:45:20 [pynccl.py:69] vLLM is using nccl==2.21.5
INFO 03-19 01:45:20 [pynccl.py:69] vLLM is using nccl==2.21.5


[W319 01:45:20.732822939 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W319 01:45:20.733028929 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 03-19 01:45:22 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1,2,3.json
(VllmWorkerProcess pid=1046552) (VllmWorkerProcess pid=1046560) (VllmWorkerProcess pid=1046555) INFO 03-19 01:45:22 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1,2,3.json
INFO 03-19 01:45:22 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1,2,3.json
INFO 03-19 01:45:22 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1,2,3.json
INFO 03-19 01:45:22 [shm_broadcast.py:258] vLLM message queue communication handle: Handle(local_reader_ranks=[1, 2, 3], buffer_handle=(3, 4194304, 6, 'psm_ea89b86a'), local_subscribe_addr='ipc:///tmp/c1bdbbc4-ff0b-48ee-af51-06501445a066', remote_subscribe_addr=None, remote_addr_ipv6=False)
INFO 03-19 01:45:22 [parallel_state.py:9

...

(VllmWorkerProcess pid=1046560) INFO 03-19 01:45:44 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(VllmWorkerProcess pid=1046555) INFO 03-19 01:45:44 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.


Capturing CUDA graph shapes:  97% 34/35 [00:20<00:00,  1.71it/s]

(VllmWorkerProcess pid=1046555) INFO 03-19 01:46:05 [custom_all_reduce.py:229] Registering 4375 cuda graph addresses


Capturing CUDA graph shapes: 100% 35/35 [00:21<00:00,  1.60it/s]

(VllmWorkerProcess pid=1046552) INFO 03-19 01:46:06 [custom_all_reduce.py:229] Registering 4375 cuda graph addresses
INFO 03-19 01:46:06 [custom_all_reduce.py:229] Registering 4375 cuda graph addresses
(VllmWorkerProcess pid=1046560) INFO 03-19 01:46:06 [custom_all_reduce.py:229] Registering 4375 cuda graph addresses
(VllmWorkerProcess pid=1046560) INFO 03-19 01:46:06 [model_runner.py:1570] Graph capturing finished in 22 secs, took 0.50 GiB
(VllmWorkerProcess pid=1046555) INFO 03-19 01:46:06 [model_runner.py:1570] Graph capturing finished in 22 secs, took 0.50 GiB
INFO 03-19 01:46:06 [model_runner.py:1570] Graph capturing finished in 22 secs, took 0.50 GiB
(VllmWorkerProcess pid=1046552) INFO 03-19 01:46:06 [model_runner.py:1570] Graph capturing finished in 22 secs, took 0.50 GiB
INFO 03-19 01:46:06 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 38.39 seconds


In [8]:
pattern = r'Educational score: (\d+)\s*$'
def extract_score(text):
    match = re.search(pattern, text)
    score = 0
    if match:
        score = int(match.group(1))

    return score

In [9]:
def classify(texts):
    messages = [[{"role": "system", "content": system_prompt},
                {"role": "user", "content": "Extract:\n" + text[:25000] + "\n\nJustification:"}] for text in texts]
    outputs = llm.chat(messages, SamplingParams(temperature=0, max_tokens=512))
    responses = [output.outputs[0].text.strip() for output in outputs]
    scores =  [extract_score(response) for response in responses]
    return {"justification-v2": responses, "classification_score_v2": scores}

In [ ]:
total_count = 1000 * 1000
num_steps = 100
step_size = total_count // num_steps
for step in range(num_steps):
    print(f"Running step: {step}")

    start_i = step * step_size
    end_i = (step+1) * step_size
    
    # cls_ds_stream = ds.take(step_size).map(classify, batched=True, batch_size=step_size, input_columns=["text"])
    # cls_ds_list = list(cls_ds_stream)

    cls_ds = ds.select(range(start_i, end_i)).map(classify, batched=True, batch_size=step_size, input_columns=["text"])

    #cls_ds = Dataset.from_list(cls_ds_list)
    cls_ds.save_to_disk(f"soft-skills-v2-cls-{step}.hf")

    #ds = ds.skip(step_size)

Parameter 'function'=<function classify at 0x7ab02c0ab880> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Running step: 0


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

INFO 03-19 01:46:07 [chat_utils.py:346] Detected the chat template content format to be 'openai'. You can set `--chat-template-content-format` to override this.



cessed prompts:   0% 0/10000 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]
cessed prompts:   0% 1/10000 [00:12<35:59:19, 12.96s/it, est. speed input: 75.63 toks/s, output: 6.33 toks/s]
cessed prompts:   0% 2/10000 [00:13<15:16:44,  5.50s/it, est. speed input: 120.85 toks/s, output: 12.84 toks/s]
cessed prompts:   0% 5/10000 [00:13<4:36:20,  1.66s/it, est. speed input: 317.21 toks/s, output: 32.10 toks/s] 
cessed prompts:   0% 11/10000 [00:13<1:36:17,  1.73it/s, est. speed input: 656.95 toks/s, output: 70.64 toks/s]
cessed prompts:   0% 15/10000 [00:14<1:00:36,  2.75it/s, est. speed input: 855.26 toks/s, output: 96.29 toks/s]
cessed prompts:   0% 20/10000 [00:14<38:56,  4.27it/s, est. speed input: 1107.62 toks/s, output: 127.28 toks/s]
cessed prompts:   0% 26/10000 [00:14<25:37,  6.49it/s, est. speed input: 1414.15 toks/s, output: 164.00 toks/s]
cessed prompts:   0% 31/10000 [00:14<20:20,  8.17it/s, est. speed input: 1679.67 toks/s, output: 192.90 toks/s]
cessed 